# 初始化

# 神经网络

In [6]:
from numpy import exp, array, random, dot
#激活函数及坡度
def sig(x):
    return 1 / (1 + exp(-x))

def sig_drv(x):
    return x * (1-x)

#生成初始神经网络
def gen_layer(x,y,nnum):
    res = [2 * random.random((len(x[0]), nnum[0])) - 1]
    if len(nnum) > 1:
        for i in range(1,len(nnum)):
            res.append(2 * random.random((nnum[i-1], nnum[i])) - 1)
    res.append(2 * random.random((nnum[-1], len(y[0]))) - 1)
    return res

#输入结果
def calc(x, nn):
    res = []
    for i in range(len(nn)):
        if i == 0:
            res.append(sig(dot(x, nn[i])))
        else:
            res.append(sig(dot(res[-1], nn[i])))
    return res

#纠正
def correct(x, y, nnum, time,prt):
    layer = gen_layer(x,y,nnum)
    for j in range(time):
        res = calc(x, layer)
        det = []
        err = []
        adj = []
        for i in range(len(layer)-1,-1,-1):
            if i == len(layer)-1:
                err.append(y - res[-1])
                det.append(err[0] * sig_drv(res[-1]))
            else:
                err.append(det[-1].dot((layer[i+1]).T))
                det.append(err[-1] * sig_drv(res[i]))
        
        det = det[::-1]
        for i in range(len(det)):
            if i == 0:
                adj.append((x.T).dot(det[i]))
            else:
                adj.append((res[i-1].T).dot(det[i]))        
        for i in range(len(layer)):
            layer[i] += adj[i]
    if prt:
        print(layer)
    return layer

x = array([[0, 0, 1], [0, 1, 1], [1, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 1], [0, 0, 0]])
y = array([[0, 1, 1, 1, 1, 0, 0]]).T
calc(x, correct(x,y,[2,5,8],1000,1))[-1]

[array([[ 4.20574789, -3.02531408],
       [ 4.21188632, -3.02560312],
       [-0.4619884 , -0.2875839 ]]), array([[ 2.08188726,  2.11553879,  0.97602543, -2.37630908, -3.20211695],
       [-1.9682197 , -0.61626925,  0.24490803,  1.6935684 ,  3.44777254]]), array([[ 0.2151342 , -0.57516527,  0.77260682,  0.24783557,  1.30082872,
         1.09495593, -0.00864318, -2.24270776],
       [-0.22595177,  0.38212313, -0.3814692 ,  0.54377013,  1.38824827,
        -0.90691801, -0.0228184 , -1.92436259],
       [ 0.20357069,  0.01019786, -0.65152365,  0.84515167, -0.03562875,
         0.18523721,  1.10005398, -0.44977384],
       [ 0.06055038,  0.20405806,  0.38527798,  0.37708366, -1.03379065,
        -0.53296165, -1.18426442,  2.62883031],
       [-0.22789909, -1.1243213 ,  0.11275618, -0.63659425, -2.10940858,
         0.69821934, -1.25455291,  4.46875754]]), array([[-0.52862642],
       [ 0.13970029],
       [-0.6632172 ],
       [ 0.26821783],
       [ 1.91358946],
       [-0.74429903],
   

array([[0.02028399],
       [0.7996896 ],
       [0.79967373],
       [0.79983409],
       [0.79982347],
       [0.80404601],
       [0.03540562]])